In [2]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Load Model Evidence

In [10]:
log_dirs = dict()
log_dirs['m5'] = 'log/fabian_sim/20210824_183823_sim_m5/'
log_dirs['m7'] = 'log/fabian_sim/20210824_183816_sim_m7/'
log_dirs['m9'] = 'log/fabian_sim/20210824_185803_sim_m9/'
log_dirs['m10'] = 'log/fabian_sim/20210824_185806_sim_m10/'

# load existing results
bf = dict()
for model in [5,7,9,10]:
    bf['m'+str(model)] = load_obj('log_lklhds', log_dirs['m'+str(model)])
#     ibis.resample_particles()
#  = ibis.particles


In [25]:
bfdf = pd.DataFrame.from_dict(bf)

bfdf['t']  = np.arange(len(bfdf))
for i in [7,9,10]:
    bfdf['m5/m'+str(i)] = bfdf['m5']/bfdf['m'+str(i)]
bfdf.drop(['m5', 'm7', 'm9', 'm10'], axis=1, inplace=True)
bfdf = bfdf.melt(id_vars='t', var_name='model')
bfdf

,t,model,value
0,0,m5/m7,1.018921e+00
1,1,m5/m7,1.018921e+00
2,2,m5/m7,9.932277e-01
3,3,m5/m7,9.932277e-01
4,4,m5/m7,-3.522767e-310
...,...,...,...
595,195,m5/m10,9.786964e-01
596,196,m5/m10,9.866207e-01
597,197,m5/m10,9.893534e-01
598,198,m5/m10,9.982329e-01


In [28]:
width = 600
height = 400
c = alt.Chart(bfdf[bfdf.t>40]).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)